# Tutorial #2:  Deploy an sound classification model in Azure Container Instance (ACI)

This tutorial is **part two of a two-part tutorial series**. In the [previous tutorial](ai-sound-major-miner-classification-part1.ipynb), you trained machine learning models and then registered a model in your workspace on the cloud.  

Now, you're ready to deploy the model as a web service in [Azure Container Instances](https://docs.microsoft.com/azure/container-instances/) (ACI). A web service is an image, in this case a Docker image, that encapsulates the scoring logic and the model itself. 

In this part of the tutorial, you use Azure Machine Learning service (Preview) to:

> * Set up your testing environment
> * Retrieve the model from your workspace
> * Test the model locally
> * Deploy the model to ACI
> * Test the deployed model

ACI is a great solution for testing and understanding the workflow. For scalable production deployments, consider using Azure Kubernetes Service. For more information, see [how to deploy and where](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).


## Prerequisites

Complete the model training in the [Tutorial #1: Train an sound classification model with Azure Machine Learning](ai-sound-major-miner-classification-part1.ipynb) notebook.  

## Set up the environment

Start by setting up a testing environment.

### Import packages

Import the Python packages needed for this tutorial.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Retrieve the model

You registered a model in your workspace in the previous tutorial. Now, load this workspace and download the model to your local directory.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config()
model=Model(ws, 'sound_clasification_model')
model.download(target_dir=os.getcwd(), exist_ok=True)

print(model)

# verify the downloaded model file
#model_file_path ='outputs/sound-classification-model.h5'
model_folder_path = 'outputs'
model_file_name = 'sound-classification-model.pkl'
model_file_path = os.path.join(model_folder_path, model_file_name)
model_file_path = os.path.join(os.getcwd(), model_file_path)

os.stat(model_file_path)

## Test model locally

Before deploying, make sure your model is working locally by:
* Loading test data
* Predicting test data
* Examining the confusion matrix

### Load test data

Load the test data from the **./data/** directory created during the training tutorial.

In [ ]:
from load_wavsounds import load_data_definition, load_wavdata

testset_name = 'sound_test'
test_folder_path = 'test'

data_def_file = 'sounddata.yml'
data_def = load_data_definition(data_def_file)
mels = data_def['fft-mels']
dataWidth = data_def['data-width']
fft=data_def['fft-freq']

# data_chunk shoud be same value for sound capturing chunk
print('loading test data...')
test_dataset = load_wavdata(test_folder_path, fft=fft, mels=mels, minimum=dataWidth)

import matplotlib.pyplot as plt
#gx = np.arange(1,data_chunk,1)

sample_size = 9
#for d in train_dataset[0]:
#    print('data shape - {}'.format(d.shape))
    
figure = plt.figure(figsize=(16,16))
for d in range(0,sample_size):
    plt.subplot(3,3,d+1)
    librosa.display.specshow(test_dataset[0][d][0],x_axis='time', y_axis='mel', fmax=test_dataset[3])
    plt.colorbar(format='%+2.0f dB')
    plt.title(test_dataset[2][d])
plt.show()


### Predict test data

Feed the test dataset to the model to get predictions.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

print('tensorflow version - '+tf.__version__)

learned_model = models.load_model(model_file_path)
print('Loaded - {}'.format(model_file_path))

tdataShape = test_dataset[0][0].shape
print('test data shape is - {}'.format(tdataShape))
test_ds = test_dataset[0].reshape(len(test_dataset[0]), dataShape[1],dataShape[2],dataShape[0])

predicted = learned_model.predict(test_ds)
print('Prediction done for test data - {}'.format(predicted.shape))

###  Examine the confusion matrix

Generate a confusion matrix to see how many samples from the test set are classified correctly. Notice the mis-classified value for the incorrect predictions.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

label_index = {}
for index,l in enumerate(test_dataset[2]):
    if l in label_index:
        if label_index[l] != test_dataset[1][index]:
            print('test data has been broken!')
            break
    else:
        label_index[l] = test_dataset[1][index]
print('label_index - {}'.format(label_index))

labeled_predicted = []
for p in predicted:
    for lk in label_index.keys():
        if p.argmax() == label_index[lk]:
            labeled_predicted.append(lk)
            break
print('labeled prediction - done.')
            
#print('test_dataset[0]  - {}'.format(test_dataset[1][10]))
conf_mx = confusion_matrix(test_dataset[2], labeled_predicted)
print(conf_mx)


Use matplotlib to display the confusion matrix as a graph. In this graph, the X axis represents the actual values, and the Y axis represents the predicted values. The color in each grid represents the error rate. The lighter the color, the higher the error rate is. For example, many 5's are mis-classified as 3's. Hence you see a bright grid at (5,3).

In [ ]:
# normalize the diagonal cells so that they don't overpower the rest of the cells when visualized
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
cax = ax.matshow(norm_conf_mx, cmap=plt.cm.bone)
ticks = np.arange(0, len(label_index.keys()), 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(ticks)
ax.set_yticklabels(ticks)
fig.colorbar(cax)
plt.ylabel('true labels', fontsize=14)
plt.xlabel('predicted values', fontsize=14)
plt.savefig('conf.png')
plt.show()

## Export trained model for Edge by hdf5 format
Please see [How to try AI on Edge](https://github.com/ms-iotkithol-jp/MicCaptureIoTSoundSample#step-2---%E3%82%AF%E3%83%A9%E3%82%A6%E3%83%89%E4%B8%8A%E3%81%A7%E3%81%AE%E9%9F%B3%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9Fai%E3%81%AE%E6%A7%8B%E7%AF%89%E3%81%A8%E5%AD%A6%E7%BF%92%E6%B8%88%E3%81%BFai%E3%81%AE%E5%88%A9%E7%94%A8)

In [ ]:
edge_folder_name = 'toedge'

if not os.path.exists(edge_folder_name):
    os.mkdir(edge_folder_name)

model_for_edge_file_path = os.path.splitext(model_file_path)[0] + '.h5'
model_for_edge_file_path = os.path.join(edge_folder_name,os.path.basename(model_for_edge_file_path) )
#print('file path of model for edge - {}'.format(model_for_edge_file_path))

learned_model.save(model_for_edge_file_path)

labels_def_file_name = 'labels.txt'
with open(os.path.join(edge_folder_name, labels_def_file_name),'w', newline='\n') as lf:
    for l in label_index.keys():
        lf.write('{}\n'.format(l))

print('Download {} and {} to AI on Edge from {}'.format(os.path.basename(model_for_edge_file_path), labels_def_file_name, edge_folder_name))

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from loadwavsounds import load_wavdata, divid_data_by_minimum_shape, load_data_definition, take_fft

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sound-classification-model.pkl')
    model = models.load(model_path)

def run(raw_data):
    data_json = json.load(raw_data)
    wavdata = np.array(data_json['data'])
    sample_rate = data_json['sample-rate']
    mels = data_json['fft-mels']
    fft = data_json['fft-freq']
    dataWidth = data_json['data-width']

    predicted_results ={}
    for channel, data in enumerate(wavdata):
        fft_data = take_fft(data, sample_rate, fft, mels)
        wav_dataset = divid_data_by_minimum_shape(fft_data, dataWidth)
        data_of_sounds = np.zeros((len(wav_dataset),1, mels, dataWidth), dtype=np.float)
        for index, wd in enumerate(wav_dataset):
            data_of_sounds[index][0] = wd
    
        dataShape = data_of_sounds[0].shape
        data_of_sounds = data_of_sounds.reshape(len(data_of_sounds),dataShape[1],dataShape[2],dataShape[0])

        predicted = self.model.predict(data_of_sounds)
        result = predicted.tolist()
        predicted_results[channel] = result

        # you can return any data type as long as it is JSON-serializable
    return predicted_results

### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
#myenv.add_conda_package("sound_classification")
myenv.add_pip_package("azureml-defaults")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "guitar-sound",  "method" : "tensorflow.keras"}, 
                                               description='Predict major minor sound classification with CNN')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Create environment object containing dependencies needed by the model using the environment file (`myenv.yml`)
1. Create inference configuration necessary to deploy the model as a web service using:
   * The scoring file (`score.py`)
   * envrionment object created in previous step
1. Deploy the model to the ACI container.
1. Get the web service HTTP endpoint.

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from sklearn.externals import joblib

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'sound-classification-svc'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

### Follwoing part has not been tested. sorry

## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model with a random sample of 30 images from the test data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 

 Since the model accuracy is high, you might have to run the following code a few times before you can see a misclassified sample.

In [ ]:
import os
import random
import json
import wave
from scipy.io.wavfile import read
from loadwavsounds import load_data_definition

data_def_file = 'sounddata.yml'

print('loading data definition...')
data_def = load_data_definition(data_def_file)

test_folder_path = 'test'
test_files = []
for lf in os.listdir(test_folder_path):
    if os.path.isdir(lf):
        for tf in os.listdir(lf):
            if os.path.isfile(tf):
                if (tf.rfind('.wav')):
                    test_files.append(os.path.join(lf,if))
random.shuffle(test_files)
print('loading test data...')

wav_file_path = test_files[0]
wv = wave.open(wav_file_path,'rb')
wvinfo = wv.getparams()
fs, data = read(wav_file_path)
sounds = []
if wvinfo.nchannels == 1:
    sounds.append(data)
else:
    tdata = data.T
    for w in tdata:
        sounds.append(w)

test_samples = json.dumps({"data": sounds, "sample-rate": fs, "fft-freq": data_def['fft-freq'], "fft-mels": data_def['fft-mels'], "data-width": data_def['data-width']})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

print(result)

You can also send raw HTTP request to test the web service.

In [ ]:
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(test_dataset[0])-1)
input_data = "{\"data\": [" + str(list(test_dataset[0][random_index])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", test_dataset[3][random_index])
print("prediction:", resp.text)

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call: